In [ ]:
# !pip install ace_tools

import pandas as pd
import numpy as np
import glob
# import ace_tools as tools

LOAD THE CSVS

In [ ]:
csv_paths = glob.glob('./*.csv')
df_list = []

Define Formulas for F1

In [ ]:
BASE_FARE = 3.00
TIME_RATE = 0.40  # per minute
DISTANCE_RATE = 0.93  # per kilometer
EFFORT_WEIGHTS = {'walking': 1.0, 'transit': 2.0, 'driving': 3.0, 'bicycling': 3.5}
X1 = 60  # weight for cost
X2 = 350  # weight for effort

Functions

In [ ]:
def calculate_cost(mode, duration_sec, distance_m):
    duration_min = duration_sec / 60
    distance_km = distance_m / 1000

    if mode == 'transit':
        base_fare = 2.5
    elif mode in ['walking', 'bicycling']:
        return 0.0
    else:
        base_fare = BASE_FARE

    return base_fare + (TIME_RATE * duration_min) + (DISTANCE_RATE * distance_km)

def apply_adjustments(mode, duration, timestamp):
    hour = pd.to_datetime(timestamp, unit='s').hour
    weekday = pd.to_datetime(timestamp, unit='s').weekday()
    is_peak_hour = (weekday < 5) and (16 <= hour <= 18)

    if mode in ['walking', 'bicycling']:
        duration *= 0.90

    if is_peak_hour:
        if mode in ['walking', 'bicycling']:
            duration *= 0.95
        elif mode == 'transit':
            duration *= 0.90
        elif mode == 'driving':
            duration *= 1.50

    return duration

def calculate_score(duration, cost, effort):
    return duration + (cost * X1) + (effort * X2)

Process all the CSVS

In [ ]:
for path in csv_paths:
    try:
        df = pd.read_csv(path)
        required_cols = {'timestamp', 'mode', 'duration_seconds', 'duration_in_traffic',
                         'origin_place', 'destination_place', 'distance_meters'}
        if required_cols.issubset(df.columns):
            df = df.copy()
            df['duration_final'] = df.apply(
                lambda row: row['duration_in_traffic'] if row['mode'] == 'driving' else row['duration_seconds'],
                axis=1
            )
            df['adjusted_duration'] = df.apply(
                lambda row: apply_adjustments(row['mode'], row['duration_final'], row['timestamp']),
                axis=1
            )
            df['cost'] = df.apply(
                lambda row: calculate_cost(row['mode'], row['adjusted_duration'], row['distance_meters']),
                axis=1
            )
            df['effort'] = df['mode'].map(EFFORT_WEIGHTS)
            df['score'] = df.apply(
                lambda row: calculate_score(row['adjusted_duration'], row['cost'], row['effort']),
                axis=1
            )
            df_list.append(df)
    except Exception as e:
        print(f"Failed to process {path}: {e}")

Combine processed dataframes

In [ ]:
final_df = pd.concat(df_list, ignore_index=True)


from IPython.display import display
display(final_df)  # or however many rows you'd like to preview

final_df.to_csv('scored_routes_output.csv', index=False)


,day,time,timestamp,origin_place,origin_lat,origin_lng,destination_place,destination_lat,destination_lng,mode,distance_meters,duration_seconds,duration_in_traffic,source,duration_final,adjusted_duration,cost,effort,score
0,8/4/2025,6:00,1754312400,Space Needle,47.620476,-122.349227,Westlake Park,47.610949,-122.337021,driving,2010,540,447.0,api,447.0,447.0,7.849300,3.0,1967.9580
1,8/4/2025,6:00,1754312400,Space Needle,47.620476,-122.349227,Westlake Park,47.610949,-122.337021,transit,1981,931,NaN,api,931.0,931.0,10.548997,2.0,2263.9398
2,8/4/2025,6:00,1754312400,Space Needle,47.620476,-122.349227,Westlake Park,47.610949,-122.337021,walking,1505,1286,NaN,api,1286.0,1157.4,0.000000,1.0,1507.4000
3,8/4/2025,6:00,1754312400,Space Needle,47.620476,-122.349227,Westlake Park,47.610949,-122.337021,bicycling,1757,490,NaN,api,490.0,441.0,0.000000,3.5,1666.0000
4,8/4/2025,7:00,1754316000,Space Needle,47.620476,-122.349227,Westlake Park,47.610949,-122.337021,driving,2010,540,470.0,api,470.0,470.0,8.002633,3.0,2000.1580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,8/10/2025,21:00,1754884800,Westlake Park,47.610949,-122.337021,Smith Tower,47.601849,-122.331854,bicycling,1350,422,NaN,api,422.0,379.8,0.000000,3.5,1604.8000
948,8/10/2025,22:00,1754888400,Westlake Park,47.610949,-122.337021,Smith Tower,47.601849,-122.331854,driving,1617,364,359.0,api,359.0,359.0,6.897143,3.0,1822.8286
949,8/10/2025,22:00,1754888400,Westlake Park,47.610949,-122.337021,Smith Tower,47.601849,-122.331854,transit,1400,284,NaN,api,284.0,284.0,5.695333,2.0,1325.7200
950,8/10/2025,22:00,1754888400,Westlake Park,47.610949,-122.337021,Smith Tower,47.601849,-122.331854,walking,1261,1042,NaN,api,1042.0,937.8,0.000000,1.0,1287.8000
